In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from collections import Counter
import re
import pickle
import json
import os
from module.file import load_json, save_json
from better_profanity import profanity
from langchain_chroma import Chroma 
from langchain_openai import OpenAIEmbeddings 
from langchain.schema import Document, AIMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory, RunnableLambda
from langchain.schema import HumanMessage
from operator import itemgetter
from textwrap import dedent
from datetime import datetime

from dotenv import load_dotenv
load_dotenv()

In [ ]:
def save_pickle(path: str, file_name: str, data: list):
    """
    데이터를 pickle 파일로 저장.
    """
    os.makedirs(path, exist_ok=True)
    with open(f"{path}/{file_name}", "wb") as f:
        pickle.dump(data, f)

def load_pickle(path: str) -> list:
    """
    pickle 파일에서 데이터 불러오기.
    """
    with open(path, "rb") as f:
        return pickle.load(f)

def save_json(path: str, file_name: str, data: list):
    """
    데이터를 json 파일로 저장.
    """
    os.makedirs(path, exist_ok=True)  # path로 디렉토리 생성
    full_path = os.path.join(path,file_name)  # file_path 생성
    with open(full_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def load_json(path: str) -> list:
    """
    json 파일에서 데이터 불러오기.
    """
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def get_ids_with_state(page_num: int, url: str) -> list:
    """
    청년 정책의 ID 수집.
    Parameters:
        page_num (int): 추출하려는 총 페이지 수
        url (str): 추출 대상의 페이지 링크를 조합할 베이스 링크
    Returns:
        policy_id_list (list): 상시, 진행중인 정책의 ID를 list로 모아 반환합니다.
    """
    policy_id_list = []
    for i in range(1, page_num + 1):
        response = requests.get(f"{url}{i}")
        soup = bs(response.text, "lxml")
        
        badges = soup.select("div.badge")
        titles = soup.select("a.tit")
        organ = soup.select("div.organ-name")
        
        for j in range(len(titles)):
            badge = badges[j].find("span", attrs={"label"}).text
            if badge in ["진행중", "상시"]:
                policy_id = titles[j].attrs["id"].replace("dtlLink_", "")
                organ_name = re.sub(r"<.*?>", "", str(organ[j].select_one("p")))
                organ_name = "세종" if organ_name == "세종 세종" else organ_name
                if policy_id not in policy_id_list:
                    policy_id_list.append([policy_id, organ_name])
    
    return policy_id_list

def formated(string: str) -> str:
    """
    HTML 태그, 이스케이프 문자, 과도한 공백 제거.
    """
    tag_format = r"<.*?>"
    string = string.replace("\n", "").replace("\t", "")
    string = re.sub(tag_format, "", str(string))
    string = string.replace("  ", "")
    return string

def crawling(policy_id_list: list, url: str, params: dict, cont_attrs: bool = True) -> list:
    """
    정책 상세 정보를 수집.
    """
    total_policy = []
    format = {"br": r"<br/>", "a": r"<a href"}
    
    for id, organ in policy_id_list:
        policy = {}
        try:
            response = requests.get(f"{url}{id}")
        except:
            response = requests.get(f"{url.replace('https', 'http')}{id}")
        
        soup = bs(response.text, "html.parser")

        # 정책 이름 추출
        title = soup.find(params["title"][0], params["title"][1]).text
        policy["정책 이름"] = title
        
        if cont_attrs:
            policy["기관"] = organ
            subtitle = soup.find("p", "doc_desc").text
            subtitle = subtitle.replace("\r", " ")
            subtitle = subtitle.strip()
            policy["요약"] = subtitle
            list_tit = soup.find_all(
                name=params["list_tit"][0], attrs=params["list_tit"][1]
            )
            list_cont = soup.find_all(
                name=params["list_cont"][0], attrs=params["list_cont"][1]
            )
        else:
            list_tit = soup.find_all(name=params["list_tit"][0])
            list_cont = soup.find_all(name=params["list_cont"][0])
        
        # 항목 내용 처리
        for i in range(len(list_tit)):
            # list_cont[i].contents = ["\n", "ㅁㅁㅁ", "\n"] 또는 ["\n\t\t\t\tㅁㅁㅁㅁ\n\t\t\t\t", "<br/>", "ㅁㅁㅁ"]과 같이 나옴
            if len(list_cont[i].contents) > 1:
                contents = []
                for j in range(len(list_cont[i].contents)):
                    content = list_cont[i].contents[j]
                    # <br/> 제거
                    if re.match(format["br"], str(content)) != None:
                        content = None
                    # url만 있는 경우 추출
                    elif re.match(format["a"], str(content)) != None:
                        content = content.attrs["href"]
                    # 그 외 공백 제거, '\n', '\t', 제거 안된 html 태그 제거
                    else:
                        content = content.text
                        content = content.strip()
                        content = formated(content)
                    # 처리 작업이 끝난 후 의미있는 요소만 contents(list)에 추가
                    if content not in [None, "\n", "", ","]:
                        # \r이 있을 경우 이를 구분자로 분할한 뒤 삽입
                        if "\r" in content:
                            content = content.split("\r")
                            for con in content:
                                contents.append(con)
                        else:
                            contents.append(content)
                if len(contents) == 1:
                    contents = "".join(contents)
            else:
                contents = list_cont[i].contents
                contents = "".join(contents)
                contents = formated(contents)

            # 동일한 요소가 contents(list)에 들어있을 경우
            if (
                isinstance(contents, list)
                and len(contents) == 2
                and contents[0] == contents[1]
            ):
                contents = set(contents)
                contents = "".join(contents)
                contents = formated(contents)
            # 정책의 항목 이름, 내용 연결
            policy[list_tit[i].text] = contents
        total_policy.append(policy)
    return total_policy

# 저장 경로 생성
DATA_DIR = "../data"
os.makedirs(DATA_DIR, exist_ok=True)

# 정책 ID 수집
URL = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifList.do?pageUnit=60&pageIndex="
policy_id_list = get_ids_with_state(59, URL)
print(f"{len(policy_id_list)}개 정책 ID 크롤링 완료")

# 저장
save_pickle(f"{DATA_DIR}", "policy_id_list.pkl", policy_id_list)
print("policy_id_list 저장 완료")

# 정책 상세 정보 크롤링
DETAIL_URL = "https://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?bizId="
params = {
    "title": ["h2", "doc_tit01 type2"],
    "list_tit": ["div", "list_tit"],
    "list_cont": ["div", "list_cont"],
}
total_policy = crawling(policy_id_list, DETAIL_URL, params, cont_attrs=True)
print(f"{len(total_policy)}개 정책 상세 정보 크롤링 완료")

# 저장
save_json(f"{DATA_DIR}", "policy.json", total_policy)
print("policy.json 저장 완료")

# 삭제 항목 정의
remove_keys = [
    "정책 번호", "신청 사이트", 
    "사업관련 참고 사이트 1", "사업관련 참고 사이트 2", "첨부파일"
]

remove_values = [
    "제한없음", "", "-", "상관없음", "□제한없음",
    "□ 제한없음","- 제한없음","-제한없음"
]

# 삭제 함수 정의
def remove_keys_from_data(data, keys):
    if isinstance(data, list):
        return [remove_keys_from_data(item, keys) for item in data]
    elif isinstance(data, dict):
        return {
            key: remove_keys_from_data(value, keys)
            for key, value in data.items()
            if key not in keys
        }
    else:
        return data

def remove_values_from_data(data):
    if isinstance(data, list):
        return [remove_values_from_data(item) for item in data if item not in remove_values]
    elif isinstance(data, dict):
        return {
            key: remove_values_from_data(value)
            for key, value in data.items()
            if value not in remove_values
        }
    else:
        return data
    
# 불러오기 
data = load_json("../data/policy.json")

# 삭제
data_cleaned_keys = remove_keys_from_data(data, remove_keys)
preprocess_data = remove_values_from_data(data_cleaned_keys)

# 저장
save_json("../data","policy_result.json", preprocess_data)

print("policy_result.json 저장 완료")

# 불러오기
data = load_json("../data/policy.json")

# 모든 텍스트 추출
def text_from_json(data):
    """
    JSON 데이터에서 모든 문자열을 추출.
    """
    texts = []
    if isinstance(data, dict):
        for value in data.values():
            texts.extend(text_from_json(value))
    elif isinstance(data, list):
        for item in data:
            texts.extend(text_from_json(item))
    elif isinstance(data, str):
        texts.append(data)
    return texts

# 단어 분리
def text_to_word(text):
    """
    텍스트를 정제하고 단어 단위로 분리.
    """
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)  # 특수문자 제거
    words = text.split()  # 공백 기준으로 단어 분리
    return words

# 자주 등장하는 단어 찾기
def get_frequent_words(words, threshold=100):
    """
    단어 목록에서 자주 등장하는 단어를 찾음.
    """
    word_counts = Counter(words)
    frequent_words = [word for word, count in word_counts.items() if count >= threshold]
    return frequent_words

# 실행
if __name__ == "__main__":

    # 불러오기
    FILE_PATH = "../data/policy_result.json"
    data = load_json(FILE_PATH)
    
    # 텍스트 추출
    texts = text_from_json(data)
    
    # 단어 분리
    all_words = []
    for text in texts:
        all_words.extend(text_to_word(text))
    
    # 50번 이상 등장하는 단어
    threshold = 100
    frequent_words = get_frequent_words(all_words, threshold)
    
    # 출력
    print("[100번 이상 등장하는 단어]")
    for i in range(0, len(frequent_words), 10):
        print(", ".join(frequent_words[i:i + 10]))

# 불러오기
data = load_json("../data/policy_result.json")

# 불용어 
stopwords = [
    "수행", "경우", "해당", "통하여", "대한", "관련","등", "및", "또는", "중인", "통해",
    "따라", "서비스", "제공", "프로그램", "참여", "따른", "대한", "해당", "관한","이용", 
    "등을", "두고"
]

# 제거 함수
def remove_text(text):
    if isinstance(text, str):
        # URL 제거
        text = re.sub(r'\bhttps?://[^\s]*\.kr\b', '', text)
        
        # 특수기호 제거 (숫자, 한글, 영어 유지)
        text = re.sub(r'[^가-힣a-zA-Z0-9\s.~%]',' ', text)
        
        # 불용어 제거
        for stopword in stopwords:
            text = text.replace(stopword, '')
        return text   
    
    return text

# 데이터 처리
def process_json(data):
    if isinstance(data, dict):
        return {key: process_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [process_json(item) for item in data]
    elif isinstance(data, str):
        return remove_text(data)
    else:
        return data


cleaned_data = process_json(data)

# 저장
save_json("../data","policy_result.json",cleaned_data)
print("policy_result.json 저장 완료")

# 불러오기
data = load_json("../data/policy_result.json")

# 문자열 병합 
def merge_values(item):
    """
    리스트, 딕셔너리, 문자열을 하나의 문자열로 병합
    """
    if isinstance(item, list):
        return " ".join(merge_values(sub_item) for sub_item in item)
    elif isinstance(item, dict):
        return " ".join(f"{key}: {merge_values(value)}" for key, value in item.items())
    elif isinstance(item, str):
        return item.strip()
    else:
        return str(item)


# 특정 키는 유지, 나머지는 병합
def restructure_policy_data(data):
    """
    '정책 이름', '기관', '요약', '정책 분야'는 유지하고 나머지는 '내용'에 병합
    """
    result = []
    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                policy = {
                    "정책 이름": item.get("정책 이름", "알 수 없음"),
                    "기관": item.get("기관", "알 수 없음"),
                    "요약": item.get("요약", "알 수 없음"),
                    "정책 분야": item.get("정책 분야", "알 수 없음")
                }

                # '내용'에 나머지 항목 병합
                remaining_content = [
                    f"{key}: {merge_values(value)}"
                    for key, value in item.items()
                    if key not in ["정책 이름", "기관", "요약", "정책 분야"]
                ]
                policy["내용"] = " ".join(remaining_content)

                result.append(policy)
    return result


# 실행
restructured_data = restructure_policy_data(data)


# 저장
save_json("../data", "policy_result.json", restructured_data)
print("policy_result.json 저장 완료")

# 불러오기
data = load_json("../data/policy_result.json")

# 지역 매핑 (정규표현식: 기관명에 포함된 지역명)
region_patterns = {
    r'경.*북.*': '경상북도',
    r'경.*남.*': '경상남도',
    r'강.*원.*': '강원도',
    r'전.*북.*': '전라북도',
    r'전.*남.*': '전라남도',
    r'충.*북.*': '충청북도',
    r'충.*남.*': '충청남도',
    r'제.*주.*': '제주특별자치도',
    r'서.*울.*': '서울특별시',
    r'부.*산.*': '부산광역시',
    r'대.*구.*': '대구광역시',
    r'대.*전.*': '대전광역시',
    r'광.*주.*': '광주광역시',
    r'울.*산.*': '울산광역시',
    r'인.*천.*': '인천광역시',
    r'세.*종.*': '세종특별자치시'
}

# 데이터 변환
def replace_institution_with_region(data):
    """
    '기관'을 '지역'으로 키를 대체하고 순서를 유지합니다.
    """
    for policy in data:
        if "기관" in policy:
            institution = policy["기관"]
            region = '전국'  # 기본값 설정
            
            # 기관명에서 지역 매칭
            for pattern, mapped_region in region_patterns.items():
                if re.search(pattern, institution):
                    region = mapped_region
                    break
            
            # '기관'을 '지역'으로 대체 (순서 유지)
            updated_policy = {}
            for key, value in policy.items():
                if key == "기관":
                    updated_policy["지역"] = region
                else:
                    updated_policy[key] = value
            
            # 기존 항목을 새로운 항목으로 교체
            policy.clear()
            policy.update(updated_policy)

# 실행행
replace_institution_with_region(data)

# 저장
save_json("../data", "policy_result.json", data)
print("policy_result.json 저장 완료")

# 불러오기
data = load_json("../data/policy_result.json")

# 띄어쓰기 제거 
def remove_spaces(data):
    """
    모든 Key에서 띄어쓰기를 제거합니다.
    """
    if isinstance(data, list):
        return [remove_spaces(item) for item in data]
    elif isinstance(data, dict):
        return {key.replace(" ", ""): remove_spaces(value) for key, value in data.items()}
    else:
        return data

# 마지막 2글자 삭제 
def modify_policy_field(data):
    """
    "정책분야"의 Value 마지막 2글자를 삭제합니다.
    """
    if isinstance(data, list):
        for item in data:
            modify_policy_field(item)
    elif isinstance(data, dict):
        if "정책분야" in data and isinstance(data["정책분야"], str):
            data["정책분야"] = data["정책분야"][:-2]  # 마지막 2글자 제거
        for key, value in data.items():
            modify_policy_field(value)

# 실행
data = remove_spaces(data)
modify_policy_field(data)

# 저장
save_json("../data", "policy_result.json", data)
print("policy_result.json 저장 완료")

In [ ]:
# 경로 설정
DIRECTORY_PATH = r"..\data"
PERSIST_DIRECTORY = r"..\data\vector_store\policy"
COLLECTION_NAME = "policy"
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
TOKENIZED_DATA_PATH = os.path.join(DIRECTORY_PATH, "policy_result.json")

# JSON 데이터 불러오기 함수
data = load_json(TOKENIZED_DATA_PATH)

# Document 객체로 변환
documents = []
for policy in data:  # 리스트의 각 항목 순회
    if isinstance(policy, dict):  # 각 항목이 딕셔너리인지 확인
    # 필요한 데이터를 병합하여 page_content 생성
        merged_text = " ".join([
            policy.get("정책이름", ""), 
            policy.get("요약", ""), 
            policy.get("내용", "")
        ])

    # Document 객체 생성 및 추가
        documents.append(
            Document(
                page_content=merged_text,
                metadata={
                    "name": policy.get("정책이름", "알 수 없음"), 
                    "region": policy.get("지역", "알 수 없음"),
                    "category": policy.get("정책분야", "알 수 없음"),
                    "source": TOKENIZED_DATA_PATH,
                },
            )
        )

# Vector Store 생성/로드
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

# Vector Store 생성/로드
if os.path.exists(PERSIST_DIRECTORY):
    vector_store = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        collection_name=COLLECTION_NAME,
        embedding_function=embedding_model,
    )
else:
    os.makedirs(PERSIST_DIRECTORY, exist_ok=True)
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY,
    )

# 저장된 파일 확인
saved_files = os.listdir(PERSIST_DIRECTORY)

# Retriever 설정 - 검색 설정
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,
        "fetch_k": 10,
        "lambda_mult": 0.2,
    },
)

# db 검색 tool
@tool
def search_policy(query: str) -> list[Document]:
    """
    Vector Store에 저장된 청년 지원 정책과 해당 정책의 정보를 검색한다.
    이 도구는 청년 지원 정책 관련 질문에 대해 실행한다.
    """
    result = retriever.invoke(query)
    return result if result else [Document(page_content="검색 결과가 없습니다.")]

# web 검색 tool
@tool
def search_web(query: str) -> list[Document]:
    """
    Web에서 청년 지원 정책과 해당 정책의 정보를 검색한다.
    이 도구는 청년 지원 정책 관련 질문에 대해 실행한다.
    """
    try:
        tavily_search = TavilySearchResults(max_results=2)
        result = tavily_search.invoke(query)
        if result:
            return [
                Document(
                    page_content=item.get("content", ""),
                    metadata={"title": item.get("title", "")},
                )
                for item in result
            ]
        else:
            return [Document(page_content="검색 결과가 없습니다.")]
    except Exception as e:
        return [Document(page_content=f"오류 발생: {str(e)}")]

# 사용자 정의 비속어 리스트 로드 함수 
def load_custom_profanity(): 
    """ 
    사용자 정의 비속어 리스트를 로드하여 better-profanity에 추가합니다. 
    """ 
    custom_words_path = r"..\data\fword_list_KOR.txt"  # 비속어 리스트 파일 경로 
    try: 
        with open(custom_words_path, "r", encoding="utf-8") as f: 
            # 사용자 정의 비속어 리스트를 set으로 저장
            custom_words = set(line.strip() for line in f.readlines()) 
        profanity.add_censor_words(custom_words)  # 사용자 정의 비속어 추가 
        return custom_words  # 추가: 비속어 리스트 반환
    except FileNotFoundError: 
        print(f"비속어 리스트 파일이 존재하지 않습니다: {custom_words_path}") 
        return set()  # 추가: 비속어 리스트가 없을 경우 빈 set 반환

# 비속어 감지 함수
def is_inappropriate_message(message):
    """ 
    better-profanity와 사용자 정의 리스트를 사용해 비속어를 감지하는 함수. 
    """ 
    custom_profanity_set = load_custom_profanity()  # 사용자 정의 비속어 리스트 로드 
    
    # 메시지를 단어 단위로 나누어 비속어 감지
    words = message.split()
    for word in words:
        if word in custom_profanity_set:  # 사용자 정의 비속어 리스트 확인
            return True
    return profanity.contains_profanity(message)  # better-profanity 확인

###################################################################################################

# 대화 기록 저장 및 불러오기 
def save_conversation_to_file(history, filename="conversation_log.json"):
    session_data = {
        "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "history": history,
    }
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
    except FileNotFoundError:
        data = {"sessions": []}
    data["sessions"].append(session_data)
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
        
# 대화 기록 불러오기 함수
def load_conversations_from_file(filename="conversation_log.json"):
    """
    대화 기록을 파일에서 불러옵니다.
    """
    try:
        with open(filename, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data.get("sessions", [])
    except FileNotFoundError:
        print(f"{filename} 파일이 존재하지 않습니다.")
        return []

# 히스토리를 JSON 직렬화 가능하게 변환
def serialize_history(history):
    serialized = []
    for message in history:
        if isinstance(message, HumanMessage):
            serialized.append({"type": "human", "content": message.content})
        elif isinstance(message, AIMessage):
            serialized.append({"type": "ai", "content": message.content})
        else:
            serialized.append({"type": "unknown", "content": str(message)})
    return serialized

# JSON 데이터를 히스토리로 역변환
def deserialize_history(serialized_history):
    history = []
    for message in serialized_history:
        if message["type"] == "human":
            history.append(HumanMessage(content=message["content"]))
        elif message["type"] == "ai":
            history.append(AIMessage(content=message["content"]))
    return history

# 세션 선택
def select_session(conversations):
    if not conversations:
        print("저장된 대화 세션이 없습니다. 새 세션이 시작됩니다.")
        return None

    print("=== 기존 대화 세션 목록 ===")
    for idx, session in enumerate(conversations):
        print(f"{idx + 1}. {session['title']} ({session['date']})")

    while True:
        choice = input(f"세션 번호를 선택하세요 (1-{len(conversations)}) 또는 'new'를 입력하여 새 세션 시작: ").strip()
        if choice == "new":
            return None
        try:
            idx = int(choice) - 1
            if 0 <= idx < len(conversations):
                return conversations[idx]
        except ValueError:
            pass
        print("잘못된 입력입니다. 다시 시도하세요.")

# 사이드바 출력 함수
def display_sidebar(conversations):
    """
    대화 목록을 사이드바 형태로 출력합니다.
    """
    print("=== 대화 히스토리 ===")
    for idx, convo in enumerate(conversations):
        print(f"{idx + 1}. {convo['title']} ({convo['date']})")

# Document 객체를 dict로 변환
def serialize_documents(documents):
    if isinstance(documents, list):
        if all(isinstance(doc, Document) for doc in documents):
            return [
                {"page_content": doc.page_content, "metadata": doc.metadata}
                for doc in documents
            ]
        elif all(isinstance(doc, dict) for doc in documents):
            return documents
    return []

# LOG 저장 함수
def save_json(file_path: str, data: list):
    """
    데이터를 json 파일로 저장.
    """
    os.makedirs(os.path.dirname(file_path), exist_ok=True)  # 디렉토리 생성
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

# LLM 구성
memory = ConversationBufferMemory(memory_key="history", return_messages=True)
prompt_template = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder("agent_scratchpad"),
        (
            "ai",
            dedent(
                """
        당신은 유능한 청년지원정책 추천 전문 AI 챗봇입니다.
        주요 목표는 사용자의 요청에 따라 알맞는 청년지원정책을 추천하는 것입니다.
        다음은 답변을 작성하기 위한 지침(guidelines)입니다:
        1. 주어진 context(데이터 및 검색 결과)를 바탕으로만 대답해주세요.
        2. 모든 답변은 학습된 정책 데이터를 바탕으로 사용자가 물어본 질문에 대한 정확한 정보만 작성하세요.
        3. 답변에 불필요한 정보는 제공하지 마세요. 
        4. 해당 데이터에 없는 내용은 검색해서 대답세요. 다만, 검색 도구(TavilySearch 등)에서도 찾을 수 없는 경우, 답변을 추측하거나 임의로 생성하지 말고 "잘 모르겠습니다."라고 답변하세요.검색으로도 정보를 찾을 수 없을 경우 답변을 추측하거나 임의로 생성하지말고, "잘 모르겠습니다."라고 답변하세요.
        5. 답변은 체계적이고, 비전문가 사용자도 이해하기 쉽게 답변을 작성하세요.
        6. 항상 최신의 정확한 정보를 제공하기 위해 노력하세요.
        7. 질문을 완전히 이해하지 못할 경우, 구체적인 질문을 다시 받을 수 있도록 사용자에게 유도 질문을 하세요.     
        8. 답변 스타일은 간결하고 논리적으로 작성하세요. 필요시, 리스트 형식으로 정리하세요.
        
        위 지침을 따라 사용자의 요청에 맞는 적절한 청년지원정책 정보를 제공합니다.
        {context}
    """
            ),
        ),
        MessagesPlaceholder("history"),
        ("human", "{question}"),
    ]
)
model = ChatOpenAI(model="gpt-4o", temperature=0)
parser = StrOutputParser()


# agent 구성
agent = create_tool_calling_agent(
    llm=model, tools=[search_policy, search_web], prompt=prompt_template
)

runnable = (
    {
        "context": RunnableLambda(lambda x: retriever.invoke(x["question"])),
        "question": itemgetter("question"),
        "history": itemgetter("history"),
    }
    | prompt_template
    | model
    | parser
)

chain = RunnableWithMessageHistory(
    runnable=runnable,
    get_session_history=lambda session_id: memory.chat_memory,
    input_messages_key="question",
    history_messages_key="history",
)

toolkit = [search_policy, search_web]

agent_executor = AgentExecutor(agent=agent, tools=toolkit, verbose=True)

# 로그 저장 경로
LOG_DIR = "../data"
LOG_FILE = "../data/response_log.json"

# 메인 로직
def main():
    # 기존 대화 불러오기
    filename = "conversation_log.json"
    conversations = load_conversations_from_file(filename)

    # 세션 선택
    session = select_session(conversations)
    if session:
        print(f"선택된 세션: {session['title']}")
        history = deserialize_history(session["history"])
    else:
        print("새로운 대화 세션이 시작됩니다.")
        history = []
    
    # 사용자 질문 입력
    while True:
        query = input("질문을 입력하세요: ").strip()
        if not query:  # 입력값이 비었을 경우
            print("유효한 질문을 입력하세요.")
            continue
    
        if is_inappropriate_message(query):  # 비속어 감지
            print("부적절한 메시지가 감지되었습니다. 다시 시도하세요.")
        else:
            break  # 비속어가 없는 경우 루프 종료

    # 히스토리 업데이트
    history.append(HumanMessage(content=query))

    # 답변 생성 (예: 에이전트 호출)
    result_from_db = search_policy.invoke(query)
    result_from_web = search_web.invoke(query)

    def combine_search_results(result_from_db: list, result_from_web: list) -> str:
        combined_context = [
            "저장된 데이터에서 찾은 정보:\n",
            *[doc.page_content for doc in result_from_db],
            "실시간 web 검색에서 확인된 정보:\n",
        ]
        if result_from_web:
            combined_context.extend(
                [f"[{idx}] {doc.metadata.get('title', '제목 없음')}: {doc.page_content}"
                 for idx, doc in enumerate(result_from_web, start=1)]
            )
        else:
            combined_context.append("web 검색 결과가 없습니다.")
        return "\n".join(combined_context)
    
    combined_context = combine_search_results(result_from_db, result_from_web)

    # LLM 입력 메세지 구성
    final_input = {
        "context": combined_context,
        "question": query,
        "history": history,  # 메모리에서 불러온 이전 대화 이력을 사용
    }
    
    # 최종 응답 생성
    final_response = agent_executor.invoke(final_input)
    print(final_response["output"])
    history.append(AIMessage(content=final_response["output"]))

    # 대화 저장
    if session:
        session["history"] = serialize_history(history)
    else:
        new_session = {
            "title": f"{query[:50]} ({datetime.now().strftime('%Y-%m-%d')})",
            "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "history": serialize_history(history),
        }
        conversations.append(new_session)

    # JSON 저장
    with open(filename, "w", encoding="utf-8") as file:
        json.dump({"sessions": conversations}, file, ensure_ascii=False, indent=4)
    print(f"대화가 {filename}에 저장되었습니다.")

    # log 저장
    log = {
        "question": query.strip(),
        "db_result": serialize_documents(result_from_db),
        "web_result": serialize_documents(result_from_web),
        "final_answer": final_response["output"]
    }
    save_json(LOG_FILE, log)
    print(f"log가 {LOG_FILE}에 저장되었습니다.")

# 실행 코드
main()

# 대화 기록 불러오기 및 사이드바 출력
conversations = load_conversations_from_file()
display_sidebar(conversations)